In [3]:
import scipy
import numpy as np
from scipy import signal
from sklearn.svm import SVC
import pickle
import sklearn
from sklearn.model_selection import KFold,LeaveOneOut,LeavePOut,ShuffleSplit # 交叉验证所需的子集划分方法
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
import scipy.io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import preprocessing

In [4]:
all_mean=[[0 for _ in range(2)] for _ in range(6)]
dimension_fb=[[1,2],[2,3],[3,4],[1,3],[1,4],[2,4]]

In [32]:
# dimension=0,valence维度.dimension=1是arousal维度


# # valence运行这个
# dimension=0
# fb_l=1
# fb_h=3

# arousal运行这个
dimension=1
fb_l=1
fb_h=4


row=0
print('第',dimension,'个维度:')


total_mean=[]
for i in tqdm(np.arange(1,33)):
    if i<=9:
        number='0'+str(i)
    else:
        number=str(i)


    path='../global_data/data_preprocessed_python/s'+number+'.dat'
    with open(path, 'rb') as file:
        subject = pickle.load(file, encoding='latin1') 
    data = subject['data']
    label = subject['labels']
    data = data[:, :32]
    label = label[:, dimension]
    label[label<5]=0
    label[label>=5]=1
    # 去基线操作，
    # 用的直接相减法，目前尚未考虑四种不同的去基线的方法对结果造成的影响
    data1=data[:,:,0:384]
    data2=data[:,:,384:8065]
    data1_tmp1=np.mean(data1,axis=2) #--------------------------------------------每个通道缩为一个值
#     print(data1_tmp1.shape)  # (40, 32)

    data1_tmp2=data1_tmp1[:,:,np.newaxis]
#     print(data1_tmp2.shape) # (40, 32, 1)

    # 这边data已经去完基线了
    data=data2-data1_tmp2  # 每个通道减去一个常数
    channel_left=[2,3]
    channel_right=[19,20]
    data_left=data[:,channel_left,:]
    data_right=data[:,channel_right,:]
    
    # =====================================================================这一步操作重要吗？
    # 去趋势操作
    data=signal.detrend(data)
    # 电影数量为40
    film_number=data.shape[0]
    # 电影长度为7680
    film_length=data.shape[2]
    band=[1,4,8,13,31]
    # 频率是128HZ
    freqence=128
    low=2*band[fb_l]/freqence;   #滤波下限
    high=2*band[fb_h]/freqence; #滤波上限
    [b,a]=signal.butter(8,[low,high],'bandpass')  #构造滤波器

    # % 对左边的信号进行滤波
    filted_data_left=signal.lfilter(b,a,data_left,axis=0)
    # % 对右边的信号进行滤波
    filted_data_right=signal.lfilter(b,a,data_right,axis=0)

    # %     先用时域算一算

    # %     计算左右边的两个通道相加，先算二维数据
    sum_channel_left_1=filted_data_left[:,0,:]+filted_data_left[:,1,:]
    sum_channel_right_1=filted_data_right[:,0,:]+filted_data_right[:,1,:]

    # % 计算左边-右边
    X_unstard=sum_channel_right_1-sum_channel_left_1
    # Min-Max标准化
    scaler=preprocessing.MinMaxScaler()
    X=scaler.fit_transform(X_unstard)
    # z-score标准化，均值为0，标准差为1
    # X=preprocessing.scale(X_unstard)
    # 留一法交叉验证
    loo=LeaveOneOut()
    clf = SVC(kernel='rbf')
    # clf.fit()
    # 下面开始留一法验证
    # cross_valid=np.arange(40)
    # for train,test in loo.split(cross_valid):
    #     X=data
    #     print("%s %s" % (train, test))

    Y=label
    # scores = cross_val_score(clf, X, Y, cv=loo)
    # scores = np.array(scores)

    # print(scores)
    # print(scores.mean())
    # 交叉验证cross_val_predict输出每个样本的预测结果
    tree_predict = cross_val_predict(clf, X, Y, cv=loo)
#     print(tree_predict.shape) # (40,)
    sub_acc = accuracy_score(Y, tree_predict)
    total_mean.append(sub_acc)
print(total_mean)
print('第',dimension,'个维度所有人的准确率为：', np.mean(total_mean))

第 1 个维度:


100%|█████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.55it/s]

[0.65, 0.65, 0.875, 0.35, 0.55, 0.5, 0.675, 0.725, 0.7, 0.475, 0.65, 0.85, 0.85, 0.725, 0.475, 0.45, 0.775, 0.45, 0.8, 0.775, 0.8, 0.6, 0.5, 0.825, 0.725, 0.775, 0.775, 0.325, 0.625, 0.675, 0.675, 0.7]
第 1 个维度所有人的准确率为： 0.6546875000000001


In [ ]:
# valence: 0.75390625
# arousal:0.68125